In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    NoAlertPresentException,
)
import time

SITE = "sdbr"
USERS = ["sdbr910903","sdbr890409","sdbr910918","sdbr890611",]
PASSWORD = "123qwe!@#"

# ===== 공용 헬퍼 =====
def wait_click(driver, css, timeout=10):
    el = WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css))
    )
    el.click()
    return el

def wait_send_keys(driver, css, keys, timeout=10, clear_first=True):
    el = WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css))
    )
    if clear_first:
        el.clear()
    el.send_keys(keys)
    return el

def accept_alert_if_any(driver, timeout=3, max_count=3):
    """최대 max_count번까지 연속 alert를 대기/수락"""
    for _ in range(max_count):
        try:
            WebDriverWait(driver, timeout).until(EC.alert_is_present())
            alert = driver.switch_to.alert
            # print("Alert text:", alert.text)  # 필요하면 확인
            alert.accept()
            # 다음 alert가 이어서 뜨는 경우를 대비해 잠깐 대기
            time.sleep(0.3)
        except (TimeoutException, NoAlertPresentException):
            break

for uid in USERS:
    driver = webdriver.Chrome()
    driver.set_window_position(0, 0)
    driver.set_window_size(1400, 900)

    try:
        # 접속
        driver.get(f"https://{SITE}.edup.co.kr/")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))

        # 로그인
        wait_send_keys(driver, "#login_id", uid)
        wait_send_keys(driver, "#login_pw", PASSWORD)
        wait_click(driver, "#smt_login")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))

        # (선택) 비번 변경 페이지 대응
        # 해당 폼이 있을 때만 처리: 존재 유무를 짧게 탐지
        time.sleep(0.5)
        try:
            pw1 = driver.find_element(By.CSS_SELECTOR, "body > div > div.center > div > form > table > tbody > tr:nth-child(2) > td > div:nth-child(1) > input")
            pw2 = driver.find_element(By.CSS_SELECTOR, "body > div > div.center > div > form > table > tbody > tr:nth-child(2) > td > div:nth-child(2) > input")
            submit = driver.find_element(By.CSS_SELECTOR, "body > div > div.center > div > form > div > input")

            new_pw = "123qwe!@#"
            pw1.clear(); pw1.send_keys(new_pw)
            pw2.clear(); pw2.send_keys(new_pw)
            submit.click()

            # 비번 변경 후 재로그인
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#login_id")))
            PASSWORD = new_pw
            wait_send_keys(driver, "#login_id", uid)
            wait_send_keys(driver, "#login_pw", PASSWORD)
            wait_click(driver, "#smt_login")
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
        except NoSuchElementException:
            print("입력창1이 없어서 건너뜀")

        # 상단 메뉴 클릭 (로그아웃/마이페이지 등 사이트 구조에 맞춰 사용)
        try:
            wait_click(driver, "body > div.center > div > div.head > div.topbar > div > ul.toplogin_wrap > li:nth-child(2) > a", timeout=5)
            # 이어서 뜨는 브라우저 alert(최대 2~3개) 수락
            accept_alert_if_any(driver, timeout=3, max_count=3)
        except TimeoutException:
            print("입력창2이 없어서 건너뜀")

        # 첫 번째 드롭다운
        wait_click(driver, "#p_id_select")
        wait_click(driver, "#p_id_select ul li:nth-child(1)")

        # 과목/회차 루프
        for i in range(1, 18):
            # 두 번째 드롭다운 열고 i번째 선택
            wait_click(driver, "#s_id_select > p")
            # li가 부족하면 중단
            try:
                wait_click(driver, f"#s_id_select ul > li:nth-child({i})", timeout=5)
            except TimeoutException:
                print(f"li {i} 없음 → 루프 종료")
                break

            # 강의 보기(예: 8번째 열 a링크)
            try:
                wait_click(driver, "body > div > div.center > div:nth-child(2) > div.lecture_wrap > div.sub_content2 > div > table > tbody > tr > td:nth-child(8) > span > a", timeout=5)
            except TimeoutException:
                print(f"{i}번째 항목의 강의 링크 없음 → 계속 진행")
                continue

            # 새 창(탭) 대기 → 이동 → 닫기
            try:
                WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                handles = driver.window_handles
                base, child = handles[0], handles[1]
                driver.switch_to.window(child)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
                time.sleep(2)  # 재생 트리거/출석 체크 등을 위한 짧은 체류
                driver.close()
                driver.switch_to.window(base)
            except TimeoutException:
                print("새 창이 열리지 않음 → 계속 진행")
                # 혹시 창이 안 열렸다면 그냥 다음 반복으로
                continue

    finally:
        # 세션 종료
        try:
            driver.quit()
        except Exception:
            pass